<a href="https://colab.research.google.com/github/erickuo5124/MLG_HW1/blob/main/DrBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 24.1MB/s 
     |████████████████████████████████| 1.5MB 31.0MB/s 
     |████████████████████████████████| 194kB 18.5MB/s 
     |████████████████████████████████| 235kB 39.2MB/s 
     |████████████████████████████████| 2.2MB 56.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 


In [2]:
import glob

ans_path = []
data_path = []
for filename in glob.glob('/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/*.txt'):
  if 'score' in filename:
    ans_path.append(filename)
  else:
    data_path.append(filename)

print(data_path)
print(ans_path)

['/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/29.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/15.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/14.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/28.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/16.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/17.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/13.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/12.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/10.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/11.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/9.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/8.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/5.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/4.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/6.txt', '/content/drive/MyDrive/MLG/h

In [3]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

import torch
import networkx as nx
from torch_geometric.data import Data, ClusterData, ClusterLoader
from torch_geometric.utils import from_networkx

torch.set_printoptions(precision=8)

ans_group = []
data_group = []
loader_group = []

for i in range(30):
  ans = {}
  with open(ans_path[i]) as f:
    line = f.readline()
    while line:
      key, value = line.split()
      key = eval(key)
      ans[key] = eval(value)
      line = f.readline()
  ans_group.append(ans)

  G = nx.read_edgelist(data_path[i], nodetype = int)
  edge_index = from_networkx(G).edge_index
  x = torch.tensor([[G.degree(node),1,1] for node in G.nodes], dtype=torch.float)
  y = torch.tensor([ans[node] for node in G.nodes], dtype=torch.float)
  data = Data(edge_index=edge_index, x=x, y=y)
  data_group.append(data)
  cluster_data = ClusterData(data, num_parts=128)
  train_loader = ClusterLoader(cluster_data, batch_size=16, shuffle=True)
  loader_group.append(train_loader)

<IPython.core.display.Javascript object>

Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!
Computing ME

In [4]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

class GCNConv(MessagePassing):
  def __init__(self, in_channels, out_channels):
    super(GCNConv, self).__init__(aggr='add')

  def forward(self, x, edge_index):
    # Compute normalization.
    row, col = edge_index
    deg = degree(col, x.size(0), dtype=x.dtype) + 1
    deg_inv_sqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    # Start propagating messages.
    return self.propagate(edge_index, x=x, norm=norm)

  def message(self, x_j, norm):
    # Normalize node features.
    return norm.view(-1, 1) * x_j

In [11]:
from torch.nn import Linear, GRUCell, ReLU, Sigmoid
import torch.nn.functional as F
from torch.nn import Sequential

class DrBC(torch.nn.Module):
  def __init__(self, num_L):
    super(DrBC, self).__init__()
    self.num_L = num_L

    self.data_in = Sequential(Linear(data.num_features, 128), ReLU())
    self.Aggregation = GCNConv(128, 128)
    self.Combine = GRUCell(128, 128, bias=False)
    self.data_out = Sequential(Linear(128, 64), ReLU(), Linear(64,1), ReLU())

  def forward(self, graph):
    # encoder
    h = self.data_in(graph.x.cuda())
    h = F.normalize(h,dim=-1,p=2)
    layer = torch.tensor([]).cuda()
    layer = torch.cat((layer, torch.unsqueeze(h, 0)))
    for _ in range(self.num_L):
      hn = self.Aggregation(h, graph.edge_index.cuda())
      h = self.Combine(h, hn)
      h = F.normalize(h,dim=-1,p=2)
      layer = torch.cat((layer, torch.unsqueeze(h, 0)))

    h, _ = torch.max(layer, 0)
    z = F.normalize(h,dim=-1,p=2)

    # decoder
    pred = self.data_out(z)
    pred = torch.squeeze(pred)

    # loss
    label = graph.y.cuda()
    pair_ids_src = torch.randint(graph.num_nodes, (graph.num_nodes * 5,)).cuda()
    pair_ids_tgt = torch.randint(graph.num_nodes, (graph.num_nodes * 5,)).cuda()

    preds = torch.index_select(pred, 0, pair_ids_src) - torch.index_select(pred, 0, pair_ids_tgt)
    labels = torch.index_select(label, 0, pair_ids_src) - torch.index_select(label, 0, pair_ids_tgt)
    preds, labels = torch.sigmoid(preds), torch.sigmoid(labels)
    loss = torch.nn.BCELoss()(preds, labels)
    return pred, loss

In [6]:
def top_n_accuracy(train, n, ans):
  num = int(len(train) * n/100)
  train = train.tolist()
  predict = {}
  for i in range(len(train)):
    predict[i] = train[i]

  predict = sorted(predict.items(), key = lambda x: x[1], reverse = True)[:num]
  ans = sorted(ans.items(), key = lambda x: x[1], reverse = True)[:num]
  ans = [item[0] for item in ans]
  hit = 0
  for i in range(num):
    if predict[i][0] in ans:
      hit += 1
  return hit/num

In [7]:
def train(i):
  model.train()

  for sub_data in loader_group[i]:
    optimizer.zero_grad()
    out, loss = model(sub_data)
    # print(out)
    # print(f'Epoch: {epoch:03d}, G_index: {index:02d}, Loss: {loss:.6f}, Accuracy: {top_n_accuracy(out, 10, ans_group[index])}')
    loss.backward()
    optimizer.step()

In [12]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = DrBC(5).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

index = 0
for epoch in range(10000):
  train(index)
  out, loss = model(data_group[index])
  print(out)
  print(f'Epoch: {epoch:03d}, G_index: {index:02d}, Loss: {loss:.6f}, Accuracy: {top_n_accuracy(out, 10, ans_group[index])}')
  index += 1 if index < 29 else -29

<IPython.core.display.Javascript object>

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 000, G_index: 00, Loss: 0.693147, Accuracy: 0.726
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 001, G_index: 01, Loss: 0.693147, Accuracy: 0.706
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 002, G_index: 02, Loss: 0.693147, Accuracy: 0.702
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 003, G_index: 03, Loss: 0.693147, Accuracy: 0.726
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 004, G_index: 04, Loss: 0.693147, Accuracy: 0.718
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 005, G_index: 05, Loss: 0.693147, Accuracy: 0.71
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Epoch: 006, G_index: 0

KeyboardInterrupt: ignored

In [ ]:
torch.save(model, '/content/drive/MyDrive/MLG/hw1/net.pkl')
torch.save(model.state_dict(), '/content/drive/MyDrive/MLG/hw1/net_params.pkl')

In [ ]:
test_path = '/content/drive/MyDrive/MLG/hw1/hw1_data/youtube/com-youtube.txt'
testans_path = '/content/drive/MyDrive/MLG/hw1/hw1_data/youtube/com-youtube_score.txt'

test_ans = {}
with open(testans_path) as f:
  line = f.readline()
  while line:
    key, value = line.split(':')
    key = eval(key)
    test_ans[key] = eval(value)
    line = f.readline()

G = nx.read_edgelist(test_path, nodetype = int)
edge_index = from_networkx(G).edge_index
x = torch.tensor([[G.degree(node),1,1] for node in G.nodes], dtype=torch.float)
y = torch.tensor([test_ans[node] for node in G.nodes], dtype=torch.float)
test_data = Data(edge_index=edge_index, x=x, y=y)
test_data

In [ ]:
out, loss = model(test_data)
print(f'Loss: {loss:.6f}, Accuracy: {top_n_accuracy(out, 5, test_ans)}')
print(out)